In [42]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import holdout
from sklearn.model_selection import cross_val_score, cross_validate
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset = pd.read_csv("data_files/Social_Network_Ads.csv")

In [4]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


# Holdout (500 primeiras linhas treino, resto validação)

In [8]:
dataset_train = dataset.iloc[:300,:]
dataset_test = dataset.iloc[300:,:]

In [9]:
print(len(dataset_train),len(dataset_test),len(dataset))

300 100 400


# Separando dados do target

In [10]:
dataset_train_x = dataset_train.iloc[:,1:-1]
print(dataset_train_x.head())

dataset_train_y = dataset_train.iloc[:,-1:]
print(dataset_train_y.head())

   Gender  Age  EstimatedSalary
0    Male   19            19000
1    Male   35            20000
2  Female   26            43000
3  Female   27            57000
4    Male   19            76000
   Purchased
0          0
1          0
2          0
3          0
4          0


In [11]:
dataset_test_x = dataset_test.iloc[:,1:-1]
print(dataset_test_x.head())

dataset_test_y = dataset_test.iloc[:,-1:]
print(dataset_test_y.head())

     Gender  Age  EstimatedSalary
300  Female   58            38000
301    Male   48            74000
302  Female   37           137000
303    Male   37            79000
304  Female   40            60000
     Purchased
300          1
301          1
302          1
303          1
304          0


# Encoding variables to numeric values

In [12]:
number = LabelEncoder()

In [13]:
dataset_train_x_encoded = pd.DataFrame(columns=dataset_train_x.columns)

In [14]:
dataset_train_x_encoded['Gender'] = number.fit_transform(dataset_train_x['Gender'])

In [15]:
dataset_train_x_encoded['Age'] = dataset_train_x['Age']

In [16]:
dataset_train_x_encoded['EstimatedSalary'] = dataset_train_x['EstimatedSalary']

In [17]:
dataset_test_x_encoded = pd.DataFrame(columns=dataset_test_x.columns, index=dataset_test_x.index)

In [18]:
dataset_test_x_encoded['Gender'] = number.fit_transform(dataset_test_x['Gender'])

In [19]:
dataset_test_x_encoded['Age'] = dataset_test_x['Age']

In [20]:
dataset_test_x_encoded['EstimatedSalary'] = dataset_test_x['EstimatedSalary']

In [21]:
model = GaussianNB()
#model.fit(dataset_train_x_encoded, dataset_train_y)
kf = KFold(n_splits=10,shuffle=False)

In [23]:
scores = cross_val_score(model, dataset_train_x_encoded, dataset_train_y, cv=kf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [24]:
scores

array([0.6       , 1.        , 0.9       , 0.96666667, 0.93333333,
       0.93333333, 0.9       , 0.76666667, 0.93333333, 0.76666667])

In [27]:
model.fit(dataset_train_x_encoded, dataset_train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [28]:
model.score(dataset_train_x_encoded, dataset_train_y)

0.8733333333333333

In [29]:
model.score(dataset_test_x_encoded, dataset_test_y)

0.77

In [34]:
def cross_validate_bayes(cv,data_x,data_y,nb_type='Gaussian'):
    scores_train= []
    scores_test= []
    acc_train = []
    acc_test = []
    #auc_train = []
    #auc_test = []
    precision_test = []
    precision_train = []
    recall_test = []
    recall_train = []
    f1_test = []
    f1_train = []
    y_pred_train = []
    y_pred_test = []
    
    targets = [] 
    for i in data_y.iloc[:,0].unique():
        targets.append(str(i))
    
    for train_idx, test_idx in cv.split(data_x):
        X_train, X_test = data_x.iloc[train_idx], data_x.iloc[test_idx] 
        y_train, y_test = data_y.iloc[train_idx], data_y.iloc[test_idx]
        #print(test_idx)
        if nb_type == 'Gaussian':
            model = GaussianNB() 
        if nb_type == 'Multinomial':
            model = MultinomialNB() 
        if nb_type == 'Bernoulli':    
            model = BernoulliNB() 
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        precision_test.append(precision_score(y_test, y_pred_test,average='micro'))
        precision_train.append(precision_score(y_train, y_pred_train,average='micro'))
        recall_test.append(recall_score(y_test, y_pred_test,average='micro'))
        recall_train.append(recall_score(y_train, y_pred_train,average='micro'))
        f1_test.append(f1_score(y_test, y_pred_test,average='micro'))
        f1_train.append(f1_score(y_train, y_pred_train,average='micro'))
        acc_test.append(accuracy_score(y_test, y_pred_test))
        acc_train.append(accuracy_score(y_train, y_pred_train))
        
        #auc_train.append(roc_auc_score(y_train, y_pred_train))
        #auc_test.append(roc_auc_score(y_test, y_pred_test))
        
    print("TRAIN :")
    print('Accuracy Mean', np.mean(acc_train))
    print('Precision Mean', np.mean(precision_train))
    print('Recall Mean', np.mean(recall_train))
    print('F1 Mean', np.mean(f1_train))
    
    print("TEST :")
    #print('Accuracy List Train: ', acc_test)
    print('Accuracy Mean', np.mean(acc_test))
    print('Precision Mean', np.mean(precision_test))
    print('Recall Mean', np.mean(recall_test))
    print('F1 Mean', np.mean(f1_test))


In [43]:
model = GaussianNB()
kf = KFold(n_splits=10,shuffle=False)

cross_validate_bayes(kf,dataset_train_x_encoded,dataset_train_y,nb_type='Gaussian')


cval = cross_val_score(model, dataset_train_x_encoded, dataset_train_y, cv=kf,scoring='accuracy')
print("Mean cval_score (Test): ", np.mean(cval))


cvalidate = cross_validate(model,dataset_train_x_encoded,dataset_train_y,cv=kf,scoring='accuracy', return_train_score=True)
print(sorted(cvalidate.keys()))
print("Train: ",np.mean(cvalidate['train_score']))
print("Test: ",np.mean(cvalidate['test_score']))

TRAIN :
Accuracy Mean 0.8781481481481481
Precision Mean 0.8781481481481481
Recall Mean 0.8781481481481481
F1 Mean 0.8781481481481481
TEST :
Accuracy Mean 0.8700000000000001
Precision Mean 0.8700000000000001
Recall Mean 0.8700000000000001
F1 Mean 0.8700000000000001
Mean cval_score (Test):  0.8700000000000001
['fit_time', 'score_time', 'test_score', 'train_score']
Train:  0.8781481481481481
Test:  0.8700000000000001
